# Impervious Surfaces

A script to calculate the % coverage of impervious surface within each small area.

## Script Setup

In [17]:
#load the libraries
suppressPackageStartupMessages(library(sf))
suppressPackageStartupMessages(library(rgdal))
suppressPackageStartupMessages(library(rgeos))
suppressPackageStartupMessages(library(raster))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(geojsonio))
suppressPackageStartupMessages(library(exactextractr))

In [14]:
#set a directory for export 
exportDirectory <- "C:/GitHub/Climate-Ireland/Vulnerability/1_InputData/1d_ImperviousSurface"

# Data

## Small Areas

In [31]:
#get the small area spatial data
smallAreaShp <- st_read("D:/Spatial_Data/CSO/Census_2016_Small_Areas/Shapefiles/Extracted/Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015/Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015.shp")
smallAreaShp <- as(smallAreaShp[c('GUID', 'COUNTYNAME')], "Spatial")

Reading layer `Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015' from data source `D:\Spatial_Data\CSO\Census_2016_Small_Areas\Shapefiles\Extracted\Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015\Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015.shp' using driver `ESRI Shapefile'
Simple feature collection with 18641 features and 21 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -10.66297 ymin: 51.4199 xmax: -5.996278 ymax: 55.44658
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs


## Impervious Surface

In [32]:
imperviousRaster <- raster('D:/Spatial_Data/Copernicus/LMS/IMD2015/Extracted/Processed/IMD_2015_20m_eu_03035_d05_IRL_WGS84.tif')

In [33]:
#convert to a simple feature
smallAreasSF <- st_as_sf(smallAreaShp)

#calculate the area of the raster cell (m2)
cellArea <- 20*20

#calculate the number of cells within each Small Area - uses a simple feature
smallAreasSF$impCount <- exact_extract(imperviousRaster, smallAreasSF, 'count')

#calculate the area (m2) in each small area of the raster data
smallAreasSF$impArea <- smallAreasSF$impCount*400

#calculate the area of impervious coverage in each small area
smallAreasSF$impCoverage <- (exact_extract(imperviousRaster, smallAreasSF, 'sum')/100)*cellArea

#calculate the percentage of each small area that is impervious
smallAreasSF$impPercent <- (smallAreasSF$impCoverage/smallAreasSF$impArea)*100

head(smallAreasSF)

  |======================================================================| 100%
  |======================================================================| 100%


GUID,COUNTYNAME,geometry,impCount,impArea,impCoverage,impPercent
4c07d11e-07e8-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.212777 5...,9949.146,3979658,11330.06,0.2846992
4c07d11e-07e9-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.206293 5...,9524.629,3809852,14473.12,0.3798868
4c07d11e-07ea-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.191036 5...,12084.806,4833922,19878.68,0.4112328
4c07d11e-1282-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.736572 5...,13288.708,5315483,28748.84,0.5408508
4c07d11e-1283-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.525214 5...,10508.062,4203225,54189.74,1.2892419
4c07d11e-1284-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.600256 5...,9094.104,3637642,59612.03,1.6387548


### Z Scores

In [34]:
#scale the percentages to z scores
smallAreasSF$impervious_pct_Z <- scale(smallAreasSF$impPercent)
head(smallAreasSF)

GUID,COUNTYNAME,geometry,impCount,impArea,impCoverage,impPercent,impervious_pct_Z
4c07d11e-07e8-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.212777 5...,9949.146,3979658,11330.06,0.2846992,-0.9942309
4c07d11e-07e9-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.206293 5...,9524.629,3809852,14473.12,0.3798868,-0.9901304
4c07d11e-07ea-851d-e053-ca3ca8c0ca7f,Cavan,MULTIPOLYGON (((-7.191036 5...,12084.806,4833922,19878.68,0.4112328,-0.9887800
4c07d11e-1282-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.736572 5...,13288.708,5315483,28748.84,0.5408508,-0.9831963
4c07d11e-1283-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.525214 5...,10508.062,4203225,54189.74,1.2892419,-0.9509571
4c07d11e-1284-851d-e053-ca3ca8c0ca7f,Donegal,MULTIPOLYGON (((-8.600256 5...,9094.104,3637642,59612.03,1.6387548,-0.9359008


# Export

In [35]:
#EXPORT
#build export path
exportPath <- paste(exportDirectory, 'imperviousSurface', sep = '/')
exportPath <- paste(exportPath, '.csv', sep = '')

# st_write(smallAreasBufferSF['GUID', 'greenBluePercentCORINE'], exportPath, driver = "CSV")
smallAreasBufferExport <- as(smallAreasSF, "Spatial")

smallAreasBufferExport <- as.data.frame(smallAreasBufferExport) %>% select(1,6,7)

names(smallAreasBufferExport) <- gsub("_pct_Z","",names(smallAreasBufferExport))

write.csv(smallAreasBufferExport, exportPath, row.names = FALSE)